In [19]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

import os.path

from nltk import sent_tokenize, wordpunct_tokenize
import string
from nltk.corpus import stopwords
stopWords = stopwords.words("english")
import re
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import spacy
nlp = spacy.load('en')
from spacy.symbols import nsubj

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from settings import *
from utils import *

In [4]:
ge, geSize = loadGloveEmbeddings(gloveFile)

Loading /Users/smeros/workspace/etc/bigFiles/glove.6B/glove.6B.50d.txt embeddings file


In [30]:
def claimExtraction(limitDocuments=10):
    query = createQuery(limitDocuments, 'web')
    documents = queryDB(query)

    
    source = ['paper', 'report', 'study', 'analysis', 'research', 'survey']
    people = ['expert', 'scientist']
    subject = source + people
    predicate = ['prove', 'demonstrate', 'reveal', 'state', 'mention', 'report', 'say', 'show', 'announce', 'claim', 'suggest']

    def word2vec(w):
        w = w.lower().strip()
        try:
            return(ge[w].reshape(1, -1))
        except:
            return np.zeros(geSize).reshape(1, -1)
    
    subjectVec = [word2vec(s) for s in subject]
    predicateVec = [word2vec(p) for p in predicate]
    
    
    def findPhrase(title, body, subjectVec, predicateVec):
        
        for s in sent_tokenize(body):
            subjectFound = predicateFound = False
            claim = ""
            for w in wordpunct_tokenize(s):
                
                if predicateFound == True:
                    claim = s
                    doc = nlp(claim)
                    #for np in doc.noun_chunks:
                     #   print(np.text, np.root.text, np.root.dep_, np.root.head.text)
                    root = [w for w in doc if w.head is w][0]
                    subject = list(root.lefts)[0]
                    print('claim:',subject, root)
                    break
                
                wVec = word2vec(w)
                
                if subjectFound == False:
                    for sVec in subjectVec:
                        if abs(cosine_similarity(sVec, wVec)) > 0.9:
                            subjectFound = True
                            break
                        
                if subjectFound == True:
                    for pVec in predicateVec:
                        if abs(cosine_similarity(pVec, wVec)) > 0.9:
                            predicateFound = True
                            break
    
    def findPhrase2(title, body, subjectVec, predicateVec):
        
        for s in sent_tokenize(body):
            subjectFound = predicateFound = False
            claim = ""
            holder = ""
            
            try:
                doc = nlp(s)
                rootVerb = [w for w in doc if w.head is w][0]
            except:
                continue
            
            wVec = word2vec(str(rootVerb))
            for pVec in predicateVec:
                if abs(cosine_similarity(pVec, wVec)) > 0.9:
                    predicateFound = True
                    break

                        
            if predicateFound == True:
                for np in doc.noun_chunks:
                    if(np.root.dep == nsubj and np.root.head == rootVerb):
                        wVec = word2vec(str(np.root.text))
                    
                        for sVec in subjectVec:    
                            if abs(cosine_similarity(sVec, wVec)) > 0.5:
                                    subjectFound = True
                                    holder = np.text
                                    break
                
            if subjectFound == True:
                    claim = s
                    print('claim:', claim)
                    print('by:', holder)
                    break
   


        

#    for s in source:
#        pairs.append((findEmbedding(s),findEmbedding(a)))
    
    documents.apply(lambda d: findPhrase2(d['title'],d['body'], subjectVec, predicateVec), axis=1)

claimExtraction(20)    

claim: Three studies show the benefits of healthier school meals.
by: Three studies
claim: Many studies show that calorie intake is higher when people dine out.
by: Many studies
claim: The findings, published in the June 24 online issue of Nature Neuroscience, suggest that an increased interest in food leads to a decreased interest in novelty and cocaine.
by: The findings
claim: Generally speaking, people believe that either genetics--if your mom or dad was obese, then you're likely to be as well--or personal choices are to blame.
by: people
claim: Some water experts say the single most important thing that Australia did was to create a new way of allocating that irrigation water.
by: Some water experts
claim: Surveys conducted over the past three decades show farmers consistently support the idea of controlling erosion in return for taxpayer support.
by: Surveys
claim: Abundant research shows milk does not improve bone health and is the biggest source of saturated (“bad”) fat in the d

In [13]:
def argumentExtraction(limitDocuments=10):
    query = createQuery(limitDocuments, 'web')
    documents = queryDB(query)
    

    def word2vec(w):
        return np.array([float(x) for x in "0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581".strip().split()])

    def vecSize():
        return len("0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581".strip().split())
    
    def body2Vec(body):
        sentences = sent_tokenize(body)
        
        sentencesVector=[]
        for s in sentences:
            s = s.translate(str.maketrans('','',string.punctuation)) #remove punctuation
            vec = np.zeros(vecSize())
            for w in wordpunct_tokenize(s):
                if len(w)!=1 and w not in stopWords:
                    vec += word2vec(w)
            sentencesVector.append(vec)
        return sentencesVector
    
    def title2Vec(title):
        title = title.translate(str.maketrans('','',string.punctuation)) #remove punctuation
        vec = np.zeros(vecSize())
        for w in wordpunct_tokenize(title):
             if len(w)!=1 and w not in stopWords:
                vec += word2vec(w)
        return vec   
    
    documents['bodyVecs'] = documents['body'].apply(lambda b: body2Vec(b))
    documents['titleVec'] = documents['title'].apply(lambda t: title2Vec(t))

    
    
    def nn(titleVec, bodyVecs):
        nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(bodyVecs)
        distances, indices = nbrs.kneighbors(titleVec.reshape(1, -1))
        print(indices.shape)
        return indices.tolist()
    
    documents['nn'] = documents.apply(lambda d: nn(d['titleVec'],d['bodyVecs']), axis=1)
        
        
    #t, b = documents.iat[3,0], documents.iat[3,1]

    #sent_text = 
    return documents
    
    #loadGloveEmbeddings(gloveFile)
    #print(t)
    #print(b)    
    #print (documents['title'].to_string())
argumentExtraction()    

(1, 3)
(1, 3)
(1, 3)
(1, 3)
(1, 3)
(1, 3)
(1, 3)
(1, 3)
(1, 3)
(1, 3)


,title,body,bodyVecs,titleVec,nn
0,"Three studies on school food: Nutrition standards work, and well","The School Nutrition Association’s bizarre opposition to the USDA’s nutrition standards for school meals has stimulated research.\n\nThree studies show the benefits of healthier school meals.\n\n1. From The Rudd Center, now at U. Conn: A press release announces publication of its new study in Childhood Obesity demonstrating that the rules have led to an increase in fruit consumption without increasing plate waste.\n\nUSDA, understandably pleased with this result, quotes Secretary Tom Vilsack in a press release:\n\nFor Congress to meddle with doctors’ recommendations and go back to less healthy meals now would not be in the best interest of our children.\n\n2. From the Harvard School of Public Health: It also sends a press release to announce its study demonstrating that an increase in consumption of fruits and vegetables is a direct result of the new USDA standards, and that these also do not increase plate waste.\n\n3. From the Union of Concerned Scientists: UCS announces a new position paper, “Lessons from the Lunchroom: Childhood Obesity, School Lunch, and the Way to a Healthier Future,” also documenting why school meals are so important to kids’ health. This report comes with an explanatory Infographic.\n\nAll of these aim to head off congressional opposition to the new standards and keep them in place.\n\nLet’s hope this science-based strategy does some good.\n\nPostscript: Dana Woldow argues that the school food scene would be much easier if schools actually got enough money to pay for what they serve and for decent wages to school food service workers.","[[5.434, 3.24584, -5.36146, 1.5821, 4.48851, -0.577941, -6.45944, -2.32206, -0.00858299, -8.5358, 3.61959, -1.91971, -7.23801, 1.90554, -0.1236235, 0.151554, 1.32652, -1.66296, -10.9759, -1.58353, -0.218413, -4.32627, -2.0176, -3.00703, -2.49353, -24.4699, -9.97698, 1.287663, -5.47625, -2.53838, 52.0923, -2.41722, -6.79731, -4.11853, 0.00769769, 0.0967837, 2.31114, -2.06661, 0.156533, -0.704899, -3.88323, -2.04737, -4.51854, -0.593281, -5.75263, 2.44205, 0.0362037, -2.39343, -1.49682, -10.21553], [2.926, 1.74776, -2.88694, 0.8519, 2.41689, -0.311199, -3.47816, -1.25034, -0.00462161, -4.5962, 1.94901, -1.03369, -3.89739, 1.02606, -0.0665665, 0.081606, 0.71428, -0.89544, -5.9101, -0.85267, -0.117607, -2.32953, -1.0864, -1.61917, -1.34267, -13.1761, -5.37222, 0.693357, -2.94875, -1.36682, 28.0497, -1.30158, -3.66009, -2.21767, 0.00414491, 0.0521143, 1.24446, -1.11279, 0.084287, -0.379561, -2.09097, -1.10243, -2.43306, -0.319459, -3.09757, 1.31495, 0.0194943, -1.28877, -0.80598, -5.50067], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [9.614, 5.74264, -9.48566, 2.7991, 7.94121, -1.022511, -11.42824, -4.10826, -0.01518529, -15.1018, 6.40389, -3.39641, -12.80571, 3.37134, -0.2187185, 0.268134, 2.34692, -2.94216, -19.4189, -2.80163, -0.386423, -7.65417, -3.5696, -5.32013, -4.41163, -43.2929, -17.65158, 2.278173, -9.68875, -4.49098, 92.1633, -4.27662, -12.02601, -7.28663, 0.01361899, 0.1712327, 4.08894, -3.65631, 0.276943, -1.247129, -6.87033, -3.62227, -7.99434, -1.049651, -10.17773, 4.32055, 0.0640527, -4.23453, -2.64822, -18.07363], [10.032, 5.99232, -9.89808, 2.9208, 8.28648, -1.066968, -11.92512, -4.28688, -0.01584552, -15.7584, 6.68232, -3.54408, -13.36248, 3.51792, -0.228228, 0.279792, 2.44896, -3.07008, -20.2632, -2.92344, -0.403224, -7.98696, -3.7248, -5.55144, -4.60344, -45.1752, -18.41904, 2.377224, -10....","[3.344, 1.99744, -3.29936, 0.9736, 2.76216, -0.355656, -3.97504, -1.42896, -0.00528184, -5.2528, 2.22744, -1.18136, -4.45416, 1.17264, -0.076076, 0.093264, 0.81632, -1.02336, -6.7544, -0.97448, -0.134408, -2.66232, -1.2416, -1.85048, -1.53448, -15.0584, -6.13